<div style="font-family: Arial, Helvetica, sans-serif;">
    <div style="display: flex;padding-top: 20px">
        <div><strong>Course:</strong> Introduction to Data Science</div>
    </div>
    <div style="display: flex;padding-top: 20px">
        <div style="padding-right: 10px;"><strong>Class:</strong> KHDL1</div>
        <div></div>
    </div>
    <div style="display: flex;padding-top: 20px">
        <div style="padding-right: 10px;"><strong>Group:</strong> 11 - HAHA</div>
    </div>
    <div>
        <div style="display: flex;padding-top: 20px">
            <div style="padding-right: 10px;"><strong>Members:</strong></div>
            <div></div>
        </div>
        <table style="font-size: 15px; display:flex;padding-top: 20px">
            <tr>
                <th>No.</th>
                <th>Student ID</th>
                <th>Name</th>
            </tr>
            <tr>
                <td>1</td>
                <td>22127008</td>
                <td style="text-align:left;">Đặng Châu Anh</td>
            </tr>
            <tr>
                <td>2</td>
                <td>22127014</td>
                <td style="text-align:left;">Nguyễn Kim Anh</td>
            </tr>
            <tr>
                <td>3</td>
                <td>22127147</td>
                <td style="text-align:left;">Đỗ Minh Huy</td>
            </tr>
            <tr>
                <td>4</td>
                <td>22127170</td>
                <td style="text-align:left;">Trần Dịu Huyền</td>
            </tr>
        </table>
    </div>
    <div style="font-size: 25px ;font-weight: 800; text-align: center;padding-top: 20px;">FINAL PROJECT</div>
    <div style="font-size: 20px ;font-weight: 800; text-align: center;padding-top: 20px;">SPOTIFY 2024 REWIND - DATA MODELING</div>
</div>

# **Table of content**
1. [Import package](#1)
2. [Load data](#2)
3. [Data preprocessing](#3)

    3.1. [Feature selection](#3.1)
    
    3.2. [Feature selection](#3.2)
    
4. [Data modeling](#4)

# **Data Modeling**
## **Import packages**
<a id="1"></a>

In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

## **Load data**
<a id="2"></a>
Read data from 9 csv data files in folder `spotify_charts_cleaned_final` into a DataFrame.


In [75]:
df = pd.read_csv('cleaned_data/spotify_charts_cleaned_final.csv')
df.head()

,rank,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,month,release_date,duration,rank_history
0,1,"Jung Kook, Latto",Seven (feat. Latto) (Explicit Ver.),BIGHIT MUSIC,1,1,29,1178606,1,2023-11-03,184.0,"[('2024/01/04', 1)]"
1,2,"Wren Evans, itsnk",Từng Quen,Universal Music Indochina,1,2,14,1087520,1,2023-12-17,175.0,"[('2024/01/11', 4), ('2024/01/25', 2)]"
2,3,"Wren Evans, itsnk",Tò Te Tí,Universal Music Indochina,3,4,7,1036739,1,2023-12-17,195.0,"[('2024/01/04', 4), ('2024/01/11', 3), ('2024/..."
3,4,"VSOUL, RPT MCK, Obito, Ronboogz, Boyzed","Buồn Hay Vui (feat. RPT MCK, Obito, Ronboogz &...",12 trái lê,3,5,6,919584,1,2023-12-25,292.0,"[('2024/01/04', 3), ('2024/01/11', 5), ('2024/..."
4,5,"Vũ., Dear Jane",Những Lời Hứa Bỏ Quên,WM Vietnam,1,3,7,856107,1,2023-12-14,236.0,"[('2024/01/04', 2), ('2024/01/25', 3)]"


# **Data Preprocessing**
<a id="3"></a>
Data is already cleaned, so we don't need to clean it again. We will do some preprocessing steps to prepare data for modeling.

## **Feature Selection**
<a id="3.1"></a>

For the purpose of this project, we will drop out unnecessary columns and keep only the columns that are useful for modeling. So as the first step, we will drop out the unnecessary columns: `source`. Because in Vietnam, we have too many music sources, and it is not necessary to keep this column.


In [76]:
df = df.drop(columns=['source'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rank            2000 non-null   int64  
 1   artist_names    2000 non-null   object 
 2   track_name      2000 non-null   object 
 3   peak_rank       2000 non-null   int64  
 4   previous_rank   2000 non-null   int64  
 5   weeks_on_chart  2000 non-null   int64  
 6   streams         2000 non-null   int64  
 7   month           2000 non-null   int64  
 8   release_date    2000 non-null   object 
 9   duration        2000 non-null   float64
 10  rank_history    2000 non-null   object 
dtypes: float64(1), int64(6), object(4)
memory usage: 172.0+ KB


As we see in `df.info()`, `release_date` is an object type, so we need to convert it to datetime type. Moreover, we also have the first date that the rank fluctuates in `rank_history`, so we need to extract the first date from `rank_history` as the `trending_date`. But we will not set two directly, we will calculate the difference between `trending_date` and `release_date` to get the `days_to_trend` column. This column will help us to know how many days it takes for a song to be on the chart after its release date. Furthermore, we will also extract the the month and year from both `release_date`, `trending_date` to get `release_month`, `release_year` and `trending_month`, `trending_year` respectively. These columns will help us to know the month and year of the release date and the trending date of a song.

In [77]:
def get_first_trending_date(row):
    date = row.split(', ')[0]
    date = date[2:-1]
    return date

# Convert release_date columns to datetime
df['release_date'] = pd.to_datetime(df['release_date'])

# Get the first trending date and convert it to datetime
df['trending_date'] = df['rank_history'].apply(lambda x: get_first_trending_date(x))
df['trending_date'] = pd.to_datetime(df['trending_date'])

Before do further steps, we will check the missing values in the dataset. If there are any missing values, we will fill them with the first date of the month of that song in `month` column for `trending_date`. Because we can't know the exact date of the song, but we can know the month of the song.

In [78]:
missing_values_trending = df['trending_date'].isna().sum()
missing_ratio_trending = missing_values_trending / len(df) * 100
print(f'Missing values for trending date: {missing_values_trending} ({missing_ratio_trending:.2f}%)')

missing_values_release = df['release_date'].isna().sum()
missing_ratio_release = missing_values_release / len(df) * 100
print(f'Missing values for release date: {missing_values_release} ({missing_ratio_release:.2f}%)')

Missing values for trending date: 222 (11.10%)
Missing values for release date: 0 (0.00%)


We found that there are some missing values in `trending_date`. So we will fill them with the first date of the month of that song in `month` column.

In [79]:
for index, row in df.iterrows():
    if pd.isna(row['trending_date']):
        trending_date = '2024' + '/' + str(row['month']) + '/' + '01'
        df.at[index, 'trending_date'] = trending_date
    
df['trending_date'].isna().sum()

0

Now, we will change `trending_date` to datetime type and extract the month and year from `release_date`, `trending_date` to get `release_month` and `release_year` columns, `trending_month` and `trending_year` columns. Then we will calculate the difference between `trending_date` and `release_date` to get the `days_to_trend` column, rename the `month` column to `current_month` to avoid confusion.

In [80]:
# Convert trending_date columns to datetime
df['trending_date'] = pd.to_datetime(df['trending_date'])

# Split release_time into year, month
df['release_year'] = df['release_date'].dt.year
df['release_month'] = df['release_date'].dt.month

# Split trending_time into year, month
df['trending_year'] = df['trending_date'].dt.year
df['trending_month'] = df['trending_date'].dt.month

# Days to trend
df['days_to_trend'] = (df['trending_date'] - df['release_date']).dt.days

# Rename `month` column to `current_month`
df = df.rename(columns={'month': 'current_month'})


Finally, we will drop out the `release_date`, `trending date` and `rank_history` columns because we don't need them anymore.

In [81]:
df = df.drop(columns=['release_date', 'trending_date', 'rank_history'])

And, this is the final dataset after feature selection.

In [82]:
df.sample(5)

,rank,artist_names,track_name,peak_rank,previous_rank,weeks_on_chart,streams,current_month,duration,release_year,release_month,trending_year,trending_month,days_to_trend
1909,110,"RPT MCK, tlinh",Chỉ Một Đêm Nữa Thôi,9,108,87,219038,10,142.0,2023,3,2023,10,217
1259,60,"ANH TRAI ""SAY HI"", Quân A.P, HURRYKNG, Hùng Huỳnh","ĐẦU ĐỘI SỪNG (feat. Quân A.P, HURRYKNG & Hùng ...",60,61,3,256458,7,214.0,2024,7,2024,7,3
345,146,"Ngọt, Mạc Mai Sương",Đá tan,55,193,34,136521,2,202.0,2022,11,2024,2,447
1064,65,"RPT Orijinn, Ronboogz",Don't Côi,3,62,80,230282,6,149.0,2022,11,2023,6,193
1738,139,"W/N, 267",id thang máy (feat. 267),10,114,9,170681,9,131.0,2024,7,2024,9,36
